# Chapter 10. Proccesing text
## Notebook for Python

### 10.1 Reading and cleaning text

In [9]:
import string

my_wiki_text = "<p><b>Communication</b> (from Latin <i>communicare</i>, meaning to share)"
print(my_wiki_text)
my_wiki_text_2 = my_wiki_text.replace('<p>',' ').replace('<b>',' ').replace('</b>',' ').replace('<i>',' ').replace('</i>',' ') #remove HTML tags
my_wiki_text_2 = "".join([l for l in my_wiki_text_2 if l not in string.punctuation]) #Remove punctuation
my_wiki_text_2 = my_wiki_text_2.lower()  #Convert to lower case
my_wiki_text_2 = " ".join(my_wiki_text_2.split()) #Remove double spaces 
print(my_wiki_text_2)

<p><b>Communication</b> (from Latin <i>communicare</i>, meaning to share)
communication from latin communicare meaning to share


In [90]:
from bs4 import BeautifulSoup 
my_wiki_text_3 = BeautifulSoup(my_wiki_text, 'html.parser')
my_wiki_text_3.get_text()

'Communication (from Latin communicare, meaning to share)'

In [22]:
import re
my_wiki_text_4 = re.sub('<.*?>', "", my_wiki_text)
print(my_wiki_text_4)

Communication (from Latin communicare, meaning to share)


In [24]:
tweets = "<@born_in_america: My second favorite color is green, I must acknowlege my friends for that!>, <@born_in_britain: My second favourite colour is red, I must aknowledge my friends for that!>"
print(tweets)
tweets_unified = re.sub("acknowlege|aknowledge" , "acknowledge", tweets)
tweets_unified = re.sub("col.+?r", "color", tweets_unified)
tweets_unified = re.sub("fav.+?rite" , "favorite", tweets_unified)
print(tweets_unified)

<@born_in_america: My second favorite color is green, I must acknowlege my friends for that!>, <@born_in_britain: My second favourite colour is red, I must aknowledge my friends for that!>
<@born_in_america: My second favorite color is green, I must acknowledge my friends for that!>, <@born_in_britain: My second favorite color is red, I must acknowledge my friends for that!>


In [89]:
users = re.compile(r'\B@[\w_]+').findall(tweets)
print(users)

['@born_in_america', '@born_in_britain']
